Script to generate cross-over and relationship functional group bandgap bin ranges 
William Riddle August 2021

Import dependencies for the script

In [1]:
import pandas as pd
import math
from heapq import nsmallest

Load the 831 stucture dataset with the bandgap values and their functional groups <br>
A different set of bandgaps and functional groups can be loaded in this section

In [2]:
all_data = pd.read_excel("CrystalData.xlsx", sheet_name="All") # Entire CHON data set, renamed to CrystalData
# fg_data = pd.read_json("preciseData.json").T.reset_index().replace('None', 0).fillna(0) # FGs From json
fg_data = pd.read_excel("FunctionalGroups.xlsx", sheet_name="Precise Functional Groups") # FGs From excel

<ipython-input-2-e83373c4d6af>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  all_data = pd.read_excel("CrystalData.xlsx", sheet_name="All") # Entire CHON data set, renamed to CrystalData
<ipython-input-2-e83373c4d6af>:3: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  fg_data = pd.read_excel("FunctionalGroups.xlsx", sheet_name="Precise Functional Groups") # FGs From excel


Peek the CHON dataset

In [3]:
all_data.loc[0:10]

,Refcode,SMILES Code,Chemical Name,Space group name,Cyclic or Aromatic Component (Y(A) Y(C) or N,a,a*,b,b*,c,...,name_cif,av_6highest_abs_hab_site_1 for GEOOPT STRUCTURES PBE (light basis set),av_abs_habs_site_1 for GEOOPT STRUCTURES PBE,av_12highest_abs_hab_site_1 for GEOOPT STRUCTURES PBE,CSD-GEOOPT 6highest_Hab PBE,RefCode,reorganization energy (meV) done with PBE used AIMS basis set on the order of triple zeta basis,name_cif.1,sum of|Habs| values GEOOPT PBE,num_neighbors_site_1
0,ABAWEG01,CC(=O)NCCNC(=O)C,"N,N'-(1,2-ethanediyl)bis(acetamide)",P-1 [CI-1],N,4.9420,4.920599,6.5960,6.535365,7.0000,...,ABAWEG01,23.540000,5.185882,14.128333,1.940000,ABAWEG01,278.796308,ABAWEG01,176.32,34.0
1,ABEJIC,c1ccc(CCc2cccc3ccccc23)c(c1)CCc4cccc5ccccc45,"1,1'-(1,2-Phenylenediethyne-2,1-diyl)dinaphtha...",P-1 [CI-1],Y(A),10.4374,10.424474,10.5171,10.554045,11.0370,...,ABEJIC,20.653333,8.010000,12.364167,5.705000,ABEJIC,167.961506,ABEJIC,152.19,19.0
2,ABEJOH,O=C1N[C@H]2[C@H](N(CN2[n+](o)[oH])[n+](o)[oH])...,"2,4,6-Trinitro-2,4,6,8-tetraazabicyclo(3.3.0)o...",P21/C [C2H-5],Y(C),11.3962,11.950050,6.4089,6.486353,12.3267,...,ABEJOH,8.015000,2.524583,4.993333,0.016667,ABEJOH,528.959878,ABEJOH,60.59,24.0
3,ABEWAG,COC,Dimethyl ether,P42/N [C4H-4],N,11.3414,11.224662,11.3414,11.224662,4.8799,...,ABEWAG,46.156667,13.212222,37.706667,-1.976667,ABEWAG,615.856978,ABEWAG,475.64,36.0
4,ACETAC07,CC(=O)O,Ethanoic acid,PNA21 [C2V-9],N,13.1510,13.194938,3.9230,3.922421,5.7620,...,ACETAC07,41.756667,10.873077,22.661667,30.603333,ACETAC07,609.429927,ACETAC07,282.70,26.0
5,ACTOLD07,CC(=O)Nc1ccc(C)cc1,1-Acetamido-4-methylbenzene,P21/C [C2H-5],Y(A),11.6620,11.644070,9.4694,9.428147,7.3997,...,ACTOLD07,22.998333,4.885278,14.128333,3.221667,ACTOLD07,434.397263,ACTOLD07,175.87,36.0
6,ADGALA03,OC[C@H]1O[C@H](O)[C@H](O)[C@@H](O)[C@H]1O,\a-D-Galactose,P212121 [D2-4],Y(C),5.8999,5.830533,7.8433,7.756198,15.6850,...,ADGALA03,22.336667,7.070000,12.758333,-5.020000,ADGALA03,734.810683,ADGALA03,155.54,22.0
7,ADUWUS01,COC(=O)c1nccnc1N,Methyl 3-aminopyrazine-2-carboxylate,P21/C [C2H-5],Y(A),6.3149,6.298270,16.5274,16.685123,6.4544,...,ADUWUS01,33.658333,14.676111,21.160833,20.233333,ADUWUS01,528.528014,ADUWUS01,264.17,18.0
8,AFAFAP,[NH3+][C@@H]1CC[C@@H]2CC[C@H](N2C1=O)c(o)[oH],6-ammonio-5-oxooctahydroindolizine-3-carboxylate,P21 [C2-2],Y(C),7.8572,7.935700,5.9533,5.944979,9.7560,...,AFAFAP,30.743333,7.899286,17.758333,-24.610000,AFAFAP,765.891261,AFAFAP,221.18,28.0
9,AFEREJ,CC1(C)O[C@@H](CC(=O)O)C(=O)O1,"(S)-(2,2-Dimethyl-5-oxo-1,3-dioxolan-4-yl)acet...",P21 [C2-2],Y(C),5.2406,5.218097,7.8236,7.798294,9.7180,...,AFEREJ,15.220000,4.327500,8.551667,3.136667,AFEREJ,476.968853,AFEREJ,103.86,24.0


Peek the functional group dataset

In [4]:
fg_data.loc[0:10]

,Unnamed: 0,Refcode,SMILES,CyclicAcetal,Alcohol,CyclicAlcohol,AromaticAlcohol,Aldehyde,Aldoxime,CyclicAldoxime,...,SecondaryKetimine,CyclicSecondaryKetimine,TertiaryAmine,CyclicTertiaryAmine,AromaticTertiaryAmine,aromaticRingCount,nonAromaticRingCount,ringCount,totalAlcohols,AminoAcid
0,ABAWEG01,ABAWEG01,CC(=O)NCCNC(C)=O,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,No
1,ABEJIC,ABEJIC,c1ccc2c(c1)cccc2C#Cc1ccccc1C#Cc1cccc2ccccc12,0,0,0,0,0,0,0,...,0,0,0,0,0,5,0,5,0,No
2,ABEJOH,ABEJOH,O=C1NC2C(N(CN2N(=O)=O)N(=O)=O)N1N(=O)=O,0,0,0,0,0,0,0,...,0,0,0,2,0,0,2,2,0,No
3,ABEWAG,ABEWAG,COC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,No
4,ACETAC07,ACETAC07,CC(O)=O,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,No
5,ACTOLD07,ACTOLD07,CC(=O)Nc1ccc(C)cc1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,No
6,ADGALA03,ADGALA03,OCC1OC(O)C(O)C(O)C1O,0,1,3,0,0,0,0,...,0,0,0,0,0,0,1,1,5,No
7,ADUWUS01,ADUWUS01,COC(=O)c1nccnc1N,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,No
8,AFAFAP,AFAFAP,[NH3+]C1CCC2CCC(N2C1=O)C(=O)[O-],0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,0,Yes
9,AFEREJ,AFEREJ,CC1(C)OC(CC(O)=O)C(=O)O1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,No


In [5]:
# The drop statement removes all non-fg and non-ring associated columns. 
# This means all functional groups in the entire set will be considered for distrubiton analysis
FG_GROUPS = fg_data.drop(["AminoAcid", "Refcode", "SMILES","Unnamed: 0"], axis = 1) # Use if fg_data from excel was used

# FG_GROUPS = fg_data.drop(["AminoAcid", "Refcode", "SMILES","index"], axis = 1) # Use if fg_data from json was used

fgs = FG_GROUPS.columns # List of column name string
print("Functional groups to be considerd: \n", fgs)

bandgap = all_data[["bandgap"]] # Select the bandgap of each structures
bandgap_fgs = pd.concat([FG_GROUPS, bandgap], axis=1) # Concatinate the bandgaps and functional groups row-wise together

Functional groups to be considerd: 
 Index(['CyclicAcetal', 'Alcohol', 'CyclicAlcohol', 'AromaticAlcohol',
       'Aldehyde', 'Aldoxime', 'CyclicAldoxime', 'Alkene', 'CyclicAlkene',
       'Alkyne', 'CyclicAlkyne', 'Amide', 'CyclicAmide', 'Azide', 'Azo',
       'CyclicAzo', 'Carbamate', 'CyclicCarbamate', 'Carbonate',
       'CyclicCarbonate', 'Carboxylate', 'CarboxylicAcid', 'Ester',
       'CyclicEster', 'Ether', 'CyclicEther', 'CyclicHemiAcetal',
       'CyclicHemiKetal', 'HydroPeroxide', 'Imide', 'CyclicImide', 'Isocynate',
       'Isonitrile', 'Ketone', 'CyclicKetone', 'Ketoxime', 'CyclicKetoxime',
       'Nitrile', 'Nitro', 'Nitroso', 'Nitrosooxy', 'CyclicPeroxide',
       'PrimaryAmine', 'CyclicPrimaryAmine', 'AromaticPrimaryAmine',
       'PrimaryKetimine', 'CyclicPrimaryKetimine', 'SecondaryAldimine',
       'CyclicSecondaryAldimine', 'SecondaryAmine', 'CyclicSecondaryAmine',
       'AromaticSecondaryAmine', 'SecondaryKetimine',
       'CyclicSecondaryKetimine', 'TertiaryAmine

View the bandgap_fgs cleaned dataset. Note that the refcodes and SMILES codes are eliminated in order to perform purley numeric computation

In [6]:
bandgap_fgs.head()

,CyclicAcetal,Alcohol,CyclicAlcohol,AromaticAlcohol,Aldehyde,Aldoxime,CyclicAldoxime,Alkene,CyclicAlkene,Alkyne,...,SecondaryKetimine,CyclicSecondaryKetimine,TertiaryAmine,CyclicTertiaryAmine,AromaticTertiaryAmine,aromaticRingCount,nonAromaticRingCount,ringCount,totalAlcohols,bandgap
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.882338
1,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,5,0,5,0,2.350893
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,2,2,0,3.715734
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.041824
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5.218628


In [7]:
slice_val = max(nsmallest(50, bandgap_fgs['bandgap']))
slice_val   

1.73679158

In [8]:
math.ceil(slice_val)

2

In [9]:
bandgap_values = [x*0.5 for x in range(
                0, 
                math.ceil(slice_val)+1
                )
] 
print("bandgap_vales = ", bandgap_values)

bandgap_vales =  [0.0, 0.5, 1.0]


Slice bandgap_fgs by 50 lowest bandgap structures

In [10]:
fifty_lowest = bandgap_fgs[bandgap_fgs['bandgap'] <= slice_val]

In [11]:
fifty_lowest

,CyclicAcetal,Alcohol,CyclicAlcohol,AromaticAlcohol,Aldehyde,Aldoxime,CyclicAldoxime,Alkene,CyclicAlkene,Alkyne,...,SecondaryKetimine,CyclicSecondaryKetimine,TertiaryAmine,CyclicTertiaryAmine,AromaticTertiaryAmine,aromaticRingCount,nonAromaticRingCount,ringCount,totalAlcohols,bandgap
30,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,3,1,4,0,1.733953
31,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,3,1,4,0,1.680069
32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1.736792
44,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,2,0,2,0,1.635647
91,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,5,0,5,0,0.805988
114,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,2,3,0,1.392820
172,0,0,0,0,0,0,0,2,0,0,...,0,0,0,1,0,2,1,3,0,1.443041
196,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,2,2,0,1.358652
227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,10,0,10,0,1.098365
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,7,0,1.541417


In [12]:
fifty_lowest = fifty_lowest.drop([col for col in fifty_lowest if sum(fifty_lowest[col]) < 1], axis=1)

In [13]:
bin_counts = pd.DataFrame(columns=fifty_lowest.columns) 

In [14]:
num_structures = []
for bg in bandgap_values: # loop over bandgap values
    # Slice a bandgap range out of the functional group and bandgap dataframe
    bg_range = fifty_lowest.loc[
        (fifty_lowest['bandgap'] >= bg) & 
        (fifty_lowest['bandgap'] <= bg+0.5)
    ]
    num_structures.append(len(bg_range))
    # Insert the total counts of each functional group within that range as the values for a row denoted by bandgap range 
    bin_counts.loc[bg] = [
        len(
            bg_range.loc[
                bg_range[f] > 0
            ]
        )
        for f in fifty_lowest.columns
    ] 


In [15]:
bin_counts

,CyclicAlcohol,AromaticAlcohol,Alkene,CyclicAlkene,Alkyne,Amide,CyclicAmide,Azo,CyclicAzo,CarboxylicAcid,...,AromaticSecondaryAmine,CyclicSecondaryKetimine,TertiaryAmine,CyclicTertiaryAmine,AromaticTertiaryAmine,aromaticRingCount,nonAromaticRingCount,ringCount,totalAlcohols,bandgap
0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.5,0,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,4,2,4,0,4
1.0,1,2,5,8,1,0,5,0,1,1,...,4,4,2,2,1,21,12,23,3,23


Append the statistical results to the reusltant matrix

In [18]:
total = bin_counts.copy(deep=True)
total.loc["total"] = list(bin_counts.sum()) 

In [22]:
writer = pd.ExcelWriter("fifty_lowest.xlsx", engine="xlsxwriter")
total.to_excel(writer, sheet_name="Matrix")
writer.save()
